This notebook looks at the case when there is one UE connected to the base station, and then the xapp is started to log the rlc buffer change overtime. 

In [7]:
from fabric import Connection


tx = Connection(
    host='pc490.emulab.net',
    user = 'PeterYao',
    port=22,
)

delay = Connection(
    host='pc500.emulab.net',
    user = 'PeterYao',
    port=22,
)


router = Connection(
    host='pc487.emulab.net',
    user='PeterYao',
    port=22,
)

rx = Connection(
    host='pc816.emulab.net',
    user
    = 'PeterYao',  
    port=22,
)

conns = [router, delay, tx, rx]


In [15]:
rx.sudo("killall iperf3")
rx.sudo("ip netns exec ue1 iperf3 -s -1 -p 4008 -D")
# run the client in the background
rx.run("iperf3 -c 12.1.1.130 -p 4008 -t 60 > /dev/null 2>&1 &", pty=False)

rx.run("/mydata/flexric/build/examples/xApp/c/monitor/xapp_gtp_mac_rlc_pdcp_moni")

[UTIL]: Setting the config -c file to /usr/local/etc/flexric/flexric.conf
[UTIL]: Setting path -p for the shared libraries to /usr/local/lib/flexric/
[xAapp]: Initializing ... 
[xApp]: nearRT-RIC IP Address = 127.0.0.1, PORT = 36422
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/libmac_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/libtc_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/libgtp_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/libkpm_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/librlc_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/librc_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/libslice_sm.so 
[E2 AGENT]: Opening plugin from path = /usr/local/lib/flexric/libpdcp_sm.so 
[NEAR-RIC]: Loading SM ID = 142 with def = MAC_STATS_V0 
[NEAR-RIC]: Loading SM ID = 146 with def = TC_STATS_V0 
[NEAR-RIC]: Loading SM ID = 148 with def = GT

<Result cmd='/mydata/flexric/build/examples/xApp/c/monitor/xapp_gtp_mac_rlc_pdcp_moni' exited=0>

In [13]:
rx.get("/tmp/xapp_db_1727450026849498")

In [14]:
import sqlite3
import pandas as pd

# Path to your SQLite file (replace 'your_database_file.sqlite' with your file)
db_file = 'xapp_db_1727449551020207'

# Connect to the SQLite database file
conn = sqlite3.connect(db_file)

# Fetch the list of tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", tables)

table_name = 'RLC_bearer'
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Close the connection after extracting the data
conn.close()

# Display the DataFrame
print(df)

# # Save DataFrame to CSV if needed
df.to_csv('xapp_db_1727449551020207.csv', index=False)


Tables in the database:              name
0          MAC_UE
1      RLC_bearer
2     PDCP_bearer
3           SLICE
4        UE_SLICE
5        GTP_NGUT
6  KPM_MeasRecord
7   KPM_LabelInfo
                 tstamp  ngran_node  mcc  mnc  mnc_digit_len  nb_id cu_du_id  \
0      1727449553033394           2  208   95              2   3584   (null)   
1      1727449553034368           2  208   95              2   3584   (null)   
2      1727449553035368           2  208   95              2   3584   (null)   
3      1727449553036366           2  208   95              2   3584   (null)   
4      1727449553037388           2  208   95              2   3584   (null)   
...                 ...         ...  ...  ...            ...    ...      ...   
59994  1727449613030381           2  208   95              2   3584   (null)   
59995  1727449613031381           2  208   95              2   3584   (null)   
59996  1727449613032382           2  208   95              2   3584   (null)   
59997  1727449